The content of this notebook:
- Getting data using the snscrap library
- Preprocessing the tweets 

In [2]:
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git


  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\bahar\appdata\local\temp\pip-req-build-jj_b54sf
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit d72b51953f0ec05ee18761ea31c1bb82f886f7a9
  Created wheel for snscrape: filename=snscrape-0.4.3.20220107.dev56+gd72b519-py3-none-any.whl size=68193 sha256=34a9c270b7cbdb94f574fd1e1e45672ce2fcabac09fbdc3d621508d2500e6768
  Stored in directory: C:\Users\bahar\AppData\Local\Temp\pip-ephem-wheel-cache-yxwyqk1f\wheels\1a\ba\e2\39fa3a11802c4a622f2efc8be3f5ff854481051d0b4c95c1fd
Successfully built snscrape


  Running command git clone -q https://github.com/JustAnotherArchivist/snscrape.git 'C:\Users\bahar\AppData\Local\Temp\pip-req-build-jj_b54sf'


In [1]:
!pip install emoji 

  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=aa1b229dca7a31d63ae5b82ffd22121e64b0e06288cf925b8658df5573069381
  Stored in directory: c:\users\bahar\appdata\local\pip\cache\wheels\0e\d1\c5\63b511b700c9da5a62bbfc758dd4a3fd9541df51b5db0694b0
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install contractions

In [1]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import nltk
import re, string
import emoji
import contractions

In [3]:

maxTweets = 5000    # This indicates the maximum number of tweets per day that we will get with this code
search_keys = '(EnjinCoin OR ENJINCOIN OR Enjincoin OR enjincoin OR Enjin_Coin OR $ENJ OR $enj)'   # Relevant key words to get Bitcoin's tweets
start_period = dt.strptime('2021-07-01',"%Y-%m-%d").date()   # we will get tweets of one month: 2022-01-01 to 2022-02-01
end_period = dt.strptime('2022-01-01',"%Y-%m-%d").date()

start_date = start_period                       # this is the starting date parameter used in TwitterSearchScraper get API
end_date = start_period + timedelta(days=1)     # this is the end date parameter used in TwitterSearchScraper get API

tweets_list = []

while end_date <= end_period:
    print(start_date, end_date)
    try:
        ## the whole argument which will be used in TwitterSearchScraper get API
        arg = search_keys + ' ' + 'lang:en ' + 'since:' + start_date.strftime('%Y-%m-%d') + ' until:' + end_date.strftime('%Y-%m-%d')
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(arg).get_items()): #TwitterSearchScraper get API
            if i>= maxTweets:
                break
            try:
                if tweet.retweetedTweet is not None:
                    retweetedTweet = tweet.retweetedTweet  
                else:
                    retweetedTweet = 'nan'
                if tweet.inReplyToTweetId is not None:
                    inReplyToTweetId = tweet.inReplyToTweetId
                else:
                    inReplyToTweetId = 'nan'
                if tweet.mentionedUsers is not None:
                    mentionedUsers = len(tweet.mentionedUsers)
                else:
                    mentionedUsers = 0   
                if tweet.quotedTweet is not None:
                    quotedTweetId = tweet.quotedTweet.id  
                else:
                    quotedTweetId = 'nan'    
                if tweet.hashtags is not None:
                    num_hashtags = len(set(tweet.hashtags))
                else:
                    num_hashtags = 0                       

                tweets_list.append([tweet.date, tweet.rawContent, tweet.likeCount, tweet.replyCount, 
                                  tweet.retweetCount, tweet.id, tweet.user.username, tweet.url, tweet.links])
            except:  
                print(tweet.id, tweet.url)
        start_date += timedelta(days=1) ## both starting and end date should be increased by one day for getting the next day's data 
        end_date += timedelta(days=1)
    except (ValueError, TypeError, RuntimeError, KeyError) as error: ## we avoid getting our code stopped by using try/except 
        print(error, tweet.id) ## we will face errors for some tweets, we can identify those tweets by this print command
        start_date += timedelta(days=1)
        end_date += timedelta(days=1)
        
    
tweets_df = pd.DataFrame(tweets_list, columns=['datetime', 'text', 'like_count', 'reply_count','retweet_count',
                                             'tweet_Id', 'username','url','outlink'])
print(len(tweets_df))
tweets_df.to_csv('EnjinCoin_tweets_2021_1.csv')
tweets_df.head()


2021-07-01 2021-07-02
2021-07-02 2021-07-03
2021-07-03 2021-07-04
2021-07-04 2021-07-05
2021-07-05 2021-07-06
2021-07-06 2021-07-07
2021-07-07 2021-07-08
2021-07-08 2021-07-09
2021-07-09 2021-07-10
2021-07-10 2021-07-11
2021-07-11 2021-07-12
2021-07-12 2021-07-13
2021-07-13 2021-07-14
2021-07-14 2021-07-15
2021-07-15 2021-07-16
2021-07-16 2021-07-17
2021-07-17 2021-07-18
2021-07-18 2021-07-19
2021-07-19 2021-07-20
2021-07-20 2021-07-21
2021-07-21 2021-07-22
2021-07-22 2021-07-23
2021-07-23 2021-07-24
2021-07-24 2021-07-25
2021-07-25 2021-07-26
2021-07-26 2021-07-27
2021-07-27 2021-07-28
2021-07-28 2021-07-29
2021-07-29 2021-07-30
2021-07-30 2021-07-31
2021-07-31 2021-08-01
2021-08-01 2021-08-02
2021-08-02 2021-08-03
2021-08-03 2021-08-04
2021-08-04 2021-08-05
2021-08-05 2021-08-06
2021-08-06 2021-08-07
2021-08-07 2021-08-08
2021-08-08 2021-08-09
2021-08-09 2021-08-10
2021-08-10 2021-08-11
2021-08-11 2021-08-12
2021-08-12 2021-08-13
2021-08-13 2021-08-14
2021-08-14 2021-08-15
2021-08-15

,datetime,text,like_count,reply_count,retweet_count,tweet_Id,username,url,outlink
0,2021-07-01 23:54:50+00:00,For sure goes to my gem radar. Don’t want to m...,0,0,0,1410748764176879626,ChimpanXTZ31,https://twitter.com/ChimpanXTZ31/status/141074...,None
1,2021-07-01 23:35:49+00:00,$CC Main net LIVE! BREAKOUT CONFIRMED https://...,0,0,1,1410743978308243459,Crypto_WB,https://twitter.com/Crypto_WB/status/141074397...,"[TextLink(text='cryptocart.cc', url='http://cr..."
2,2021-07-01 23:31:51+00:00,I’ve found out a hidden pearl on our huge DEFI...,0,0,0,1410742978763624450,ChimpanXTZ31,https://twitter.com/ChimpanXTZ31/status/141074...,None
3,2021-07-01 23:31:29+00:00,$CC Main net LIVE! BREAKOUT CONFIRMED https://...,1,2,0,1410742889592672257,Crypto_WB,https://twitter.com/Crypto_WB/status/141074288...,"[TextLink(text='cryptocart.cc', url='http://cr..."
4,2021-07-01 23:12:46+00:00,@maid_coin Awaiting the success\n@HamzatR60663...,1,0,0,1410738179196342281,Lummy_1,https://twitter.com/Lummy_1/status/14107381791...,None


In [5]:
tweets_df.isnull().sum()

datetime                0
url                     0
text                    0
like_count              0
reply_count             0
retweet_count           0
num_uniq_hashtags       0
tweet_Id                0
username                0
displayname             0
verified                0
statusesCount           0
total _like_count       0
followers               0
followings              0
inReplyToTweetId        0
quotedTweetId           0
retweetedTweet          0
mentionedUsers          0
source_Label            0
outlink              2352
dtype: int64

In [6]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [2]:
punct = []
punct += list(string.punctuation)
punct += ['’', '==']  ## adding two other punctuations to the default list

print(punct)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '’', '==']


In [3]:
def remove_punctuations(text):
    '''
    A function to replace the punctuations with a space character
    '''
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text


def remove_emoji(text):
    '''
    A function to remove emojies from the text
    '''
    regex_pattern = re.compile(pattern = "["     
          u"\U0001F600-\U0001F64F" 
          u"\U0001F300-\U0001F5FF"
          u"\U0001F680-\U0001F6FF"
          u"\U0001F1E0-\U0001F1FF"
                            "]+", flags = re.UNICODE)
    text =  regex_pattern.sub(r'', text) 
    return text 

def expand(text):
    '''
    A function to expand contractions. For example converting "can't" to "can not"
    '''
    expanded_words = []   
    for word in text.split():
        expanded_words.append(contractions.fix(word))  
    
    expanded_text = ' '.join(expanded_words)
    return expanded_text

def ReplaceThreeOrMore(text):
    '''
    A function to look for three or more repetitions of any character, including newlines.
    For example converting "Heeeeeey" to "Heey"
    '''
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
    return pattern.sub(r"\1\1", text)

##  We will not remove the stop words because the default stop words list contains some words that could be important in financial contexts. 
   ## Another alternative approach is to customize the stop words list for our applications (not included in this notebook).   

In [31]:
def clean(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower()  #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)  #remove links and mentions
    text = re.sub(r'[$][A-Za-z][\S]*', "", text)  #remove ticker symbols ($)
    text = re.sub(r'#', '', text) #remove hashtags (#)
#     text = remove_emoji(text)   #remove emojies (it is better to replace them with the text they represent)
    text = emoji.demojize(text)     # replace any emojis with the text they represent
    text = expand(text)
    text = remove_punctuations(text) 
    text = ReplaceThreeOrMore(text)
    text = re.sub(r'\w*\d\w*', '', text).strip() # remove words that contain number
    text = ' '.join( [w for w in text.split() if len(w)>1] ) # remove words with 1 character
    text = re.sub("\s\s+" , " ", text)  # remove multiple spaces
    return text


'yaay you are lucky. this will totally explode :exploding_head: june via bitcoin'

In [19]:
tweet = "😳 Tom .....Braaaaaady g bay's Bullish on Bitcoin &amp;     Cathie Wood of Ark Invest Applies For Bitcoin ETF LOL you're - June 30th 2021 https://t.co/0lpxVB3P0A via @YouTube @bitcoin #Bitcoin #BTC $BTC 🐦"
clean(tweet)

":flushed_face: tom ..braady bay's bullish on bitcoin &amp; cathie wood of ark invest applies for bitcoin etf lol you are june via bitcoin btc :bird:"

In [21]:
text : "@Rookie b Yaaaaay  you're lucky. this will totally explode 🤯 June 3rd https://t.co/0l via @YouTube #Bitcoin $BTC"
clean(text)

In [14]:
## Clean all texts 
tweets_df["cleaned_tweet"] = tweets_df['text'].apply(lambda text: clean(text))

In [17]:
tweets_df[['text', 'cleaned_tweet']].head()

,text,cleaned_tweet
0,Gaming tokens about to explode 👀\n#gaming #Gam...,gaming tokens about to explode eyes gaming gam...
1,0.4MOT TOKENS IN #LATOKEN airdrop and maybe m...,tokens in latoken airdrop and maybe more bitco...
2,#DYDX - long alert 💸🤑\n\nExchange : FTX\nM...,dydx long alert money with wings money mouth f...
3,Every Time the Bell Rings The Titano makes MON...,every time the bell rings the titano makes mon...
4,If your interested in losing money in 2022 kee...,if your interested in losing money in keep you...


In [ ]:
tweets_df.to_excel('Bitcoin_preprocessed.xlsx')